In [0]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import os
import time
import re, string

# Load packages
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

%matplotlib inline

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# combined_trainingdata_20181117.tsv

#https://drive.google.com/open?id=121uUZ4qKIgF0By62te16JzttRnuX1dz7

if 1==1:
  #2. Get the file
  downloaded = drive.CreateFile({'id':'121uUZ4qKIgF0By62te16JzttRnuX1dz7'}) # replace the id with id of file you want to access
  downloaded.GetContentFile('combined_trainingdata_20181117.tsv')  

  #3. Read file as panda dataframe
  import pandas as pd
  df = pd.read_csv('combined_trainingdata_20181117.tsv',sep='\t') 
  df.drop(labels=['Unnamed: 0'],axis=1,inplace=True)
  df.drop(df.loc[df['complaint']=='new '].index, inplace=True)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Prep the Data

In [0]:
#make simple tsv file just in case

df[['CATEGORY_MAIN', 'CATEGORY_SUB','complaint']].to_csv('dummy_data.tsv',sep='\t')

In [5]:
!ls

adc.json  combined_trainingdata_20181117.tsv  dummy_data.tsv  sample_data


### drop any complaints in categories under 100 counts

In [0]:
df.dropna(axis=0,subset=['complaint'],inplace=True)
df=df[df.groupby('CATEGORY_SUB').CATEGORY_MAIN.transform(len) > 99]

### split

In [0]:
df_split_train, df_split_test=train_test_split( df, test_size=0.2, random_state=412, stratify=df.CATEGORY_SUB)

In [8]:
print(df_split_train.shape)
print(df_split_test.shape)
df_split_train[['CATEGORY_MAIN', 'CATEGORY_SUB','complaint']].to_csv('df_split_train.tsv',sep='\t')
df_split_test[['CATEGORY_MAIN', 'CATEGORY_SUB','complaint']].to_csv('df_split_test.tsv',sep='\t')

(768085, 11)
(192022, 11)


### split_train txt file creation

In [9]:
import csv

output=open('fasttext_train.txt','w')



with open('df_split_train.tsv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:  # don't write column names
            line_count+=1
        else:
            #print(f'\t{row[3]} is in group {row[1]} categorized as {row[2]}.')
            output.write('__label__'+row[2]+' '+row[3]+' \n')  #__label__subcategory complaint \n
            line_count += 1
    print(f'Processed {line_count} lines.')
    
output.close()

Processed 768086 lines.


split_train_main txt file creation

In [10]:
import csv

output=open('fasttext_train_main.txt','w')



with open('df_split_train.tsv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:  # don't write column names
            line_count+=1
        else:
            #print(f'\t{row[3]} is in group {row[1]} categorized as {row[2]}.')
            output.write('__label__'+row[1]+' '+row[3]+' \n')  #__label__subcategory complaint \n
            line_count += 1
    print(f'Processed {line_count} lines.')
    
output.close()

Processed 768086 lines.


df_test txt file creation

In [11]:
import csv

output=open('fasttext_test.txt','w')



with open('df_split_test.tsv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:  # don't write column names
            line_count+=1
        else:
            #print(f'\t{row[3]} is in group {row[1]} categorized as {row[2]}.')
            output.write(row[3]+' \n')  #__label__subcategory complaint \n
            line_count += 1
    print(f'Processed {line_count} lines.')
    
output.close()

Processed 192023 lines.


# Install fasttext

In [12]:
!wget https://github.com/facebookresearch/fastText/archive/v0.1.0.zip
!unzip v0.1.0.zip
%cd fastText-0.1.0
!make

--2018-11-29 22:39:38--  https://github.com/facebookresearch/fastText/archive/v0.1.0.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.1.0 [following]
--2018-11-29 22:39:38--  https://codeload.github.com/facebookresearch/fastText/zip/v0.1.0
Resolving codeload.github.com (codeload.github.com)... 192.30.253.120, 192.30.253.121
Connecting to codeload.github.com (codeload.github.com)|192.30.253.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.1.0.zip’

v0.1.0.zip              [ <=>                ]  92.06K  --.-KB/s    in 0.04s   

2018-11-29 22:39:39 (2.28 MB/s) - ‘v0.1.0.zip’ saved [94267]

Archive:  v0.1.0.zip
431c9e2a9b5149369cc60fb9f5beba58dcf8ca17
   creating: fastText-0.1.0/
  inflating: fastText-0.

In [13]:
!./fasttext

usage: fasttext <command> <args>

The commands supported by fasttext are:

  supervised              train a supervised classifier
  quantize                quantize a model to reduce the memory usage
  test                    evaluate a supervised classifier
  predict                 predict most likely labels
  predict-prob            predict most likely labels with probabilities
  skipgram                train a skipgram model
  cbow                    train a cbow model
  print-word-vectors      print word vectors given a trained model
  print-sentence-vectors  print sentence vectors given a trained model
  nn                      query for nearest neighbors
  analogies               query for analogies



In [14]:
!wc ../../content/fasttext_train.txt

  768085  7204766 69190378 ../../content/fasttext_train.txt


In [0]:
! cat ../../content/fasttext_train.txt >  complaint.train
! cat ../../content/fasttext_train_main.txt >  complaint.train_main
! cat ../../content/fasttext_test.txt >  complaint.test

In [16]:
!./fasttext supervised -input complaint.train -output model_complaint -lr 0.1 -epoch 25 -wordNgrams 2

Read 7M words
Number of words:  123907
Number of labels: 51
Progress: 100.0%  words/sec/thread: 395272  lr: 0.000000  loss: 0.290047  eta: 0h0m 


In [17]:
!./fasttext predict model_complaint.bin complaint.test 1 > fasttext_predict.txt

df_fasttext_predict=pd.read_csv('fasttext_predict.txt',header=None)
df_fasttext_predict.columns=['fasttext_predict']

df_fasttext_predict_word=pd.read_csv('complaint.test',header=None)
df_fasttext_predict_word.columns=['fasttext_complaint']

dummy_predict=pd.concat([df_fasttext_predict,df_fasttext_predict_word], axis=1)

dummy=df_split_test[['CATEGORY_MAIN', 'CATEGORY_SUB','complaint']].reset_index()
dummy.drop(labels=['index'],axis=1,inplace=True)

df_fasttext_predict=pd.concat([dummy.CATEGORY_SUB,dummy.complaint,dummy_predict], axis=1)
display(df_fasttext_predict.head(2))
df_fasttext_predict.fasttext_predict=df_fasttext_predict.fasttext_predict.map(lambda x:x.replace('__label__',''))
df_fasttext_predict.head(20)

print('accuracy score:',accuracy_score(df_fasttext_predict.CATEGORY_SUB, df_fasttext_predict.fasttext_predict, normalize=True, sample_weight=None))
print(classification_report(df_fasttext_predict.CATEGORY_SUB,df_fasttext_predict.fasttext_predict))

,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.822390142796138
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.57      0.39      0.46        54
     environment_abandoned_site       0.71      0.58      0.64       833
  environment_abandoned_vehicle       0.80      0.84      0.81      5043
      environment_air_pollution       0.87      0.89      0.88      3551
           environment_asbestos       0.92      0.92      0.92       542
        environment_dead_animal       0.88      0.90      0.89      1972
            environment_dumping       0.66      0.69      0.67      7219
 environment_garbage_collection       0.92      0.96      0.94     47212
            environment_general       0.65      0.48      0.55      1511
 environment_hazardous_material       0.78      0.74      0.76      1151
             environment_litter       0.65      0.58      0.62      2981
         environment_overgrowth       0.80      0.85      0.83     15871
          enviro

In [0]:
epoch=25
lr=0.1
ngram=2
df_name = 'df_fasttext_predict_sub_epoch'+str(epoch)+'_lr'+str(lr)+'_ngram'+str(ngram)+'.tsv'

df_fasttext_predict.to_csv(df_name,sep='\t')

In [23]:
foo='ls'
!$foo

args.o						  model_complaint.bin
classification-example.sh			  model_complaint.vec
classification-results.sh			  model.o
complaint.test					  PATENTS
complaint.train					  pretrained-vectors.md
complaint.train_main				  productquantizer.o
CONTRIBUTING.md					  qmatrix.o
df_fasttext_predict_sub_epoch25_lr0.1_ngram2.tsv  quantization-example.sh
dictionary.o					  quantization-results.sh
eval.py						  README.md
fasttext					  src
fasttext.o					  tutorials
fasttext_predict.txt				  utils.o
get-wikimedia.sh				  vector.o
LICENSE						  wikifil.pl
Makefile					  word-vector-example.sh
matrix.o


In [27]:
import time

start_time=time.time()
exec('foo1_'+'a = 24')
foo1_a
time.sleep(1)
print('elapsed time:',round(time.time()-start_time,1), 'seconds')


elapsed time: 1.0 seconds


# Grid Search

In [28]:
# first just simple cycle through n-gram length
for epoch in [5,10]:
  for lr in [0.1,0.5,0.9]:
    for ngram in [1,2,3]:
      print('epochs: ',epoch,'\nLearning Rate: ',lr,'\nwordNgrams: ',ngram)
      start_time=time.time()
      # build model
      print('build model: ')
      !./fasttext supervised -input complaint.train -output model_complaint -lr $lr -epoch $epoch -wordNgrams $ngram
      
      # test
      print('test model: ')
      !./fasttext predict model_complaint.bin complaint.test 1 > fasttext_predict.txt

      df_fasttext_predict=pd.read_csv('fasttext_predict.txt',header=None)
      df_fasttext_predict.columns=['fasttext_predict']

      df_fasttext_predict_word=pd.read_csv('complaint.test',header=None)
      df_fasttext_predict_word.columns=['fasttext_complaint']

      dummy_predict=pd.concat([df_fasttext_predict,df_fasttext_predict_word], axis=1)

      dummy=df_split_test[['CATEGORY_MAIN', 'CATEGORY_SUB','complaint']].reset_index()
      dummy.drop(labels=['index'],axis=1,inplace=True)

      df_fasttext_predict=pd.concat([dummy.CATEGORY_SUB,dummy.complaint,dummy_predict], axis=1)
      display(df_fasttext_predict.head(2))
      df_fasttext_predict.fasttext_predict=df_fasttext_predict.fasttext_predict.map(lambda x:x.replace('__label__',''))
      df_fasttext_predict.head(20)
      
      # evaluate
      print('accuracy score:',accuracy_score(df_fasttext_predict.CATEGORY_SUB, df_fasttext_predict.fasttext_predict, normalize=True, sample_weight=None))
      print(classification_report(df_fasttext_predict.CATEGORY_SUB,df_fasttext_predict.fasttext_predict))
      print('elapsed time:',round(time.time()-start_time,1), 'seconds')
      print('\n \n')
  
      # save predictions to tsv and output to gdrive
    
      from pydrive.auth import GoogleAuth
      from pydrive.drive import GoogleDrive
      from google.colab import auth
      from oauth2client.client import GoogleCredentials
      
      df_name = 'df_fasttext_predict_sub_epoch'+str(epoch)+'_lr'+str(lr)+'_ngram'+str(ngram)+'.tsv'

      # 1. Authenticate and create the PyDrive client.
      auth.authenticate_user()
      gauth = GoogleAuth()
      gauth.credentials = GoogleCredentials.get_application_default()
      drive = GoogleDrive(gauth)

      df_fasttext_predict.to_csv(df_name,sep='\t')

      # 2. Create & upload a file text file.
      uploaded = drive.CreateFile({'title': df_name})
      uploaded.SetContentFile(df_name)
      uploaded.Upload()
      print('Uploaded file with ID {}'.format(uploaded.get('id')))

      # print(class_report)


epochs:  5 
Learning Rate:  0.1 
wordNgrams:  1
build model: 
Read 7M words
Number of words:  123907
Number of labels: 51
Progress: 100.0%  words/sec/thread: 481178  lr: 0.000000  loss: 0.788144  eta: 0h0m 
test model: 


,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8133130578787847


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.59      0.19      0.28        54
     environment_abandoned_site       0.71      0.57      0.63       833
  environment_abandoned_vehicle       0.78      0.84      0.81      5043
      environment_air_pollution       0.85      0.89      0.87      3551
           environment_asbestos       0.92      0.92      0.92       542
        environment_dead_animal       0.88      0.91      0.89      1972
            environment_dumping       0.66      0.66      0.66      7219
 environment_garbage_collection       0.91      0.95      0.93     47212
            environment_general       0.64      0.43      0.52      1511
 environment_hazardous_material       0.78      0.72      0.75      1151
             environment_litter       0.64      0.54      0.58      2981
         environment_overgrowth       0.78      0.87      0.82     15871
          environment_recycling       0.94      0.

...

accuracy score: 0.7590276114195248
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.06      0.15      0.08        54
     environment_abandoned_site       0.45      0.53      0.48       833
  environment_abandoned_vehicle       0.75      0.77      0.76      5043
      environment_air_pollution       0.81      0.83      0.82      3551
           environment_asbestos       0.73      0.88      0.80       542
        environment_dead_animal       0.81      0.86      0.84      1972
            environment_dumping       0.60      0.58      0.59      7219
 environment_garbage_collection       0.92      0.91      0.91     47212
            environment_general       0.42      0.42      0.42      1511
 environment_hazardous_material       0.67      0.69      0.68      1151
             environment_litter       0.52      0.51      0.51      2981
         environment_overgrowth       0.78      0.75      0.77     15871
          envir

,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8191665538323734
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.53      0.35      0.42        54
     environment_abandoned_site       0.73      0.57      0.64       833
  environment_abandoned_vehicle       0.80      0.83      0.81      5043
      environment_air_pollution       0.87      0.88      0.87      3551
           environment_asbestos       0.92      0.92      0.92       542
        environment_dead_animal       0.89      0.89      0.89      1972
            environment_dumping       0.65      0.69      0.67      7219
 environment_garbage_collection       0.92      0.95      0.94     47212
            environment_general       0.64      0.48      0.55      1511
 environment_hazardous_material       0.77      0.72      0.75      1151
             environment_litter       0.64      0.59      0.61      2981
         environment_overgrowth       0.81      0.85      0.83     15871
          envir

,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8259001572736457
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.53      0.31      0.40        54
     environment_abandoned_site       0.75      0.58      0.65       833
  environment_abandoned_vehicle       0.80      0.85      0.82      5043
      environment_air_pollution       0.87      0.89      0.88      3551
           environment_asbestos       0.91      0.92      0.91       542
        environment_dead_animal       0.88      0.91      0.89      1972
            environment_dumping       0.66      0.71      0.68      7219
 environment_garbage_collection       0.92      0.96      0.94     47212
            environment_general       0.67      0.47      0.56      1511
 environment_hazardous_material       0.79      0.73      0.76      1151
             environment_litter       0.69      0.58      0.63      2981
         environment_overgrowth       0.81      0.87      0.84     15871
          envir

In [29]:
# first just simple cycle through n-gram length
for epoch in [10,25,50]:
  for lr in [0.05,0.1]:
    for ngram in [3,4]:
      print('epochs: ',epoch,'\nLearning Rate: ',lr,'\nwordNgrams: ',ngram)
      start_time=time.time()
      # build model
      print('build model: ')
      !./fasttext supervised -input complaint.train -output model_complaint -lr $lr -epoch $epoch -wordNgrams $ngram
      
      # test
      print('test model: ')
      !./fasttext predict model_complaint.bin complaint.test 1 > fasttext_predict.txt

      df_fasttext_predict=pd.read_csv('fasttext_predict.txt',header=None)
      df_fasttext_predict.columns=['fasttext_predict']

      df_fasttext_predict_word=pd.read_csv('complaint.test',header=None)
      df_fasttext_predict_word.columns=['fasttext_complaint']

      dummy_predict=pd.concat([df_fasttext_predict,df_fasttext_predict_word], axis=1)

      dummy=df_split_test[['CATEGORY_MAIN', 'CATEGORY_SUB','complaint']].reset_index()
      dummy.drop(labels=['index'],axis=1,inplace=True)

      df_fasttext_predict=pd.concat([dummy.CATEGORY_SUB,dummy.complaint,dummy_predict], axis=1)
      display(df_fasttext_predict.head(2))
      df_fasttext_predict.fasttext_predict=df_fasttext_predict.fasttext_predict.map(lambda x:x.replace('__label__',''))
      df_fasttext_predict.head(20)
      
      # evaluate
      print('accuracy score:',accuracy_score(df_fasttext_predict.CATEGORY_SUB, df_fasttext_predict.fasttext_predict, normalize=True, sample_weight=None))
      print(classification_report(df_fasttext_predict.CATEGORY_SUB,df_fasttext_predict.fasttext_predict))
      print('elapsed time:',round(time.time()-start_time,1), 'seconds')
      print('\n \n')
  
      # save predictions to tsv and output to gdrive
    
      from pydrive.auth import GoogleAuth
      from pydrive.drive import GoogleDrive
      from google.colab import auth
      from oauth2client.client import GoogleCredentials
      
      df_name = 'df_fasttext_predict_sub_epoch'+str(epoch)+'_lr'+str(lr)+'_ngram'+str(ngram)+'.tsv'

      # 1. Authenticate and create the PyDrive client.
      auth.authenticate_user()
      gauth = GoogleAuth()
      gauth.credentials = GoogleCredentials.get_application_default()
      drive = GoogleDrive(gauth)

      df_fasttext_predict.to_csv(df_name,sep='\t')

      # 2. Create & upload a file text file.
      uploaded = drive.CreateFile({'title': df_name})
      uploaded.SetContentFile(df_name)
      uploaded.Upload()
      print('Uploaded file with ID {}'.format(uploaded.get('id')))

      # print(class_report)


epochs:  10 
Learning Rate:  0.05 
wordNgrams:  3
build model: 
Read 7M words
Number of words:  123907
Number of labels: 51
Progress: 100.0%  words/sec/thread: 363681  lr: 0.000000  loss: 0.521997  eta: 0h0m 
test model: 


,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8274937246773807


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.85      0.20      0.33        54
     environment_abandoned_site       0.75      0.60      0.66       833
  environment_abandoned_vehicle       0.79      0.86      0.82      5043
      environment_air_pollution       0.86      0.89      0.87      3551
           environment_asbestos       0.91      0.91      0.91       542
        environment_dead_animal       0.88      0.92      0.90      1972
            environment_dumping       0.67      0.70      0.69      7219
 environment_garbage_collection       0.92      0.96      0.94     47212
            environment_general       0.66      0.48      0.55      1511
 environment_hazardous_material       0.79      0.71      0.75      1151
             environment_litter       0.69      0.60      0.64      2981
         environment_overgrowth       0.81      0.87      0.84     15871
          environment_recycling       0.95      0.

...

accuracy score: 0.8278999281332348
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.77      0.37      0.50        54
     environment_abandoned_site       0.76      0.60      0.67       833
  environment_abandoned_vehicle       0.79      0.86      0.82      5043
      environment_air_pollution       0.87      0.89      0.88      3551
           environment_asbestos       0.92      0.92      0.92       542
        environment_dead_animal       0.88      0.92      0.90      1972
            environment_dumping       0.67      0.70      0.69      7219
 environment_garbage_collection       0.92      0.96      0.94     47212
            environment_general       0.67      0.47      0.55      1511
 environment_hazardous_material       0.78      0.73      0.75      1151
             environment_litter       0.71      0.58      0.64      2981
         environment_overgrowth       0.81      0.87      0.84     15871
          envir

,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8277593192446698
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.53      0.31      0.40        54
     environment_abandoned_site       0.74      0.59      0.66       833
  environment_abandoned_vehicle       0.80      0.86      0.83      5043
      environment_air_pollution       0.87      0.89      0.88      3551
           environment_asbestos       0.92      0.92      0.92       542
        environment_dead_animal       0.88      0.92      0.90      1972
            environment_dumping       0.67      0.70      0.68      7219
 environment_garbage_collection       0.92      0.96      0.94     47212
            environment_general       0.68      0.48      0.56      1511
 environment_hazardous_material       0.78      0.73      0.75      1151
             environment_litter       0.69      0.59      0.63      2981
         environment_overgrowth       0.81      0.87      0.84     15871
          envir

,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8285977648394455
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.74      0.37      0.49        54
     environment_abandoned_site       0.76      0.58      0.66       833
  environment_abandoned_vehicle       0.79      0.86      0.82      5043
      environment_air_pollution       0.87      0.89      0.88      3551
           environment_asbestos       0.92      0.92      0.92       542
        environment_dead_animal       0.88      0.92      0.90      1972
            environment_dumping       0.67      0.70      0.68      7219
 environment_garbage_collection       0.92      0.96      0.94     47212
            environment_general       0.67      0.47      0.55      1511
 environment_hazardous_material       0.80      0.72      0.76      1151
             environment_litter       0.71      0.58      0.64      2981
         environment_overgrowth       0.81      0.88      0.84     15871
          envir

# Next grid search

y = 1.477827974·10-5 epoch + 1.078001467·10-2 LR - 2.994448667·10-4 nGram + 8.276153095·10-1

so pick 50 epoch, 0.1 LR, 3 wordNgram

In [30]:
epoch=50
lr=0.1
ngram=3

# first just simple cycle through n-gram length
for dim in [50,100,200]:
  for loss in ['ns', 'hs', 'softmax']:
    
    print('epochs: ',epoch,'\nLearning Rate: ',lr,'\nwordNgrams: ',ngram,'\ndimensions:',dim,'loss type:',loss)
    start_time=time.time()
    # build model
    print('build model: ')
    !./fasttext supervised -input complaint.train -output model_complaint -lr $lr -epoch $epoch -wordNgrams $ngram -dim $dim -loss $loss

    # test
    print('test model: ')
    !./fasttext predict model_complaint.bin complaint.test 1 > fasttext_predict.txt

    df_fasttext_predict=pd.read_csv('fasttext_predict.txt',header=None)
    df_fasttext_predict.columns=['fasttext_predict']

    df_fasttext_predict_word=pd.read_csv('complaint.test',header=None)
    df_fasttext_predict_word.columns=['fasttext_complaint']

    dummy_predict=pd.concat([df_fasttext_predict,df_fasttext_predict_word], axis=1)

    dummy=df_split_test[['CATEGORY_MAIN', 'CATEGORY_SUB','complaint']].reset_index()
    dummy.drop(labels=['index'],axis=1,inplace=True)

    df_fasttext_predict=pd.concat([dummy.CATEGORY_SUB,dummy.complaint,dummy_predict], axis=1)
    display(df_fasttext_predict.head(2))
    df_fasttext_predict.fasttext_predict=df_fasttext_predict.fasttext_predict.map(lambda x:x.replace('__label__',''))
    df_fasttext_predict.head(20)

    # evaluate
    print('accuracy score:',accuracy_score(df_fasttext_predict.CATEGORY_SUB, df_fasttext_predict.fasttext_predict, normalize=True, sample_weight=None))
    print(classification_report(df_fasttext_predict.CATEGORY_SUB,df_fasttext_predict.fasttext_predict))
    print('elapsed time:',round(time.time()-start_time,1), 'seconds')
    print('\n \n')

    # save predictions to tsv and output to gdrive

    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    df_name = 'df_fasttext_predict_sub_epoch'+str(epoch)+'_lr'+str(lr)+'_ngram'+str(ngram)+'_dim'+str(dim)+'_loss'+str(loss)+'.tsv'

    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    df_fasttext_predict.to_csv(df_name,sep='\t')

    # 2. Create & upload a file text file.
    uploaded = drive.CreateFile({'title': df_name})
    uploaded.SetContentFile(df_name)
    uploaded.Upload()
    print('Uploaded file with ID {}'.format(uploaded.get('id')))

    # print(class_report)


epochs:  50 
Learning Rate:  0.1 
wordNgrams:  3 
dimensions: 50 loss type: ns
build model: 
Read 7M words
Number of words:  123907
Number of labels: 51
Progress: 100.0%  words/sec/thread: 847385  lr: 0.000000  loss: 0.215249  eta: 0h0m 
test model: 


,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8285613106831509
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.47      0.37      0.41        54
     environment_abandoned_site       0.71      0.61      0.66       833
  environment_abandoned_vehicle       0.81      0.85      0.83      5043
      environment_air_pollution       0.88      0.89      0.88      3551
           environment_asbestos       0.91      0.94      0.92       542
        environment_dead_animal       0.88      0.92      0.90      1972
            environment_dumping       0.66      0.70      0.68      7219
 environment_garbage_collection       0.93      0.95      0.94     47212
            environment_general       0.65      0.50      0.56      1511
 environment_hazardous_material       0.76      0.76      0.76      1151
             environment_litter       0.66      0.61      0.63      2981
         environment_overgrowth       0.82      0.86      0.84     15871
          envir

,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8069075418441637
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.53      0.31      0.40        54
     environment_abandoned_site       0.64      0.58      0.61       833
  environment_abandoned_vehicle       0.78      0.83      0.80      5043
      environment_air_pollution       0.84      0.89      0.86      3551
           environment_asbestos       0.90      0.91      0.90       542
        environment_dead_animal       0.86      0.89      0.88      1972
            environment_dumping       0.65      0.68      0.66      7219
 environment_garbage_collection       0.93      0.94      0.94     47212
            environment_general       0.60      0.47      0.53      1511
 environment_hazardous_material       0.75      0.68      0.71      1151
             environment_litter       0.59      0.56      0.58      2981
         environment_overgrowth       0.81      0.85      0.83     15871
          envir

...

accuracy score: 0.8287748278843049
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.45      0.35      0.40        54
     environment_abandoned_site       0.71      0.62      0.66       833
  environment_abandoned_vehicle       0.80      0.85      0.83      5043
      environment_air_pollution       0.88      0.89      0.88      3551
           environment_asbestos       0.91      0.93      0.92       542
        environment_dead_animal       0.88      0.92      0.90      1972
            environment_dumping       0.67      0.71      0.69      7219
 environment_garbage_collection       0.93      0.95      0.94     47212
            environment_general       0.63      0.50      0.56      1511
 environment_hazardous_material       0.78      0.76      0.77      1151
             environment_litter       0.66      0.61      0.63      2981
         environment_overgrowth       0.82      0.87      0.84     15871
          envir

,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8067617252189854
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.44      0.30      0.36        54
     environment_abandoned_site       0.64      0.59      0.61       833
  environment_abandoned_vehicle       0.78      0.82      0.80      5043
      environment_air_pollution       0.84      0.89      0.86      3551
           environment_asbestos       0.90      0.90      0.90       542
        environment_dead_animal       0.86      0.89      0.88      1972
            environment_dumping       0.65      0.68      0.66      7219
 environment_garbage_collection       0.93      0.94      0.94     47212
            environment_general       0.60      0.46      0.52      1511
 environment_hazardous_material       0.75      0.68      0.71      1151
             environment_litter       0.59      0.57      0.58      2981
         environment_overgrowth       0.81      0.85      0.83     15871
          envir

,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,__label__environment_garbage_collection,missed white goods pick mon two weeks refriger...
1,street_repair,pothole growing years many cars use driveway t...,__label__street_repair,pothole growing years many cars use driveway t...


accuracy score: 0.8280301215485726
                                 precision    recall  f1-score   support

        environemnt_dead_animal       0.55      0.31      0.40        54
     environment_abandoned_site       0.75      0.59      0.66       833
  environment_abandoned_vehicle       0.80      0.86      0.83      5043
      environment_air_pollution       0.87      0.89      0.88      3551
           environment_asbestos       0.92      0.92      0.92       542
        environment_dead_animal       0.88      0.92      0.90      1972
            environment_dumping       0.67      0.70      0.69      7219
 environment_garbage_collection       0.92      0.96      0.94     47212
            environment_general       0.67      0.48      0.56      1511
 environment_hazardous_material       0.79      0.73      0.76      1151
             environment_litter       0.69      0.59      0.64      2981
         environment_overgrowth       0.81      0.87      0.84     15871
          envir

# Test best config

In [31]:
!ls *.tsv

df_fasttext_predict_sub_epoch10_lr0.05_ngram3.tsv
df_fasttext_predict_sub_epoch10_lr0.05_ngram4.tsv
df_fasttext_predict_sub_epoch10_lr0.1_ngram1.tsv
df_fasttext_predict_sub_epoch10_lr0.1_ngram2.tsv
df_fasttext_predict_sub_epoch10_lr0.1_ngram3.tsv
df_fasttext_predict_sub_epoch10_lr0.1_ngram4.tsv
df_fasttext_predict_sub_epoch10_lr0.5_ngram1.tsv
df_fasttext_predict_sub_epoch10_lr0.5_ngram2.tsv
df_fasttext_predict_sub_epoch10_lr0.5_ngram3.tsv
df_fasttext_predict_sub_epoch10_lr0.9_ngram1.tsv
df_fasttext_predict_sub_epoch10_lr0.9_ngram2.tsv
df_fasttext_predict_sub_epoch10_lr0.9_ngram3.tsv
df_fasttext_predict_sub_epoch25_lr0.05_ngram3.tsv
df_fasttext_predict_sub_epoch25_lr0.05_ngram4.tsv
df_fasttext_predict_sub_epoch25_lr0.1_ngram2.tsv
df_fasttext_predict_sub_epoch25_lr0.1_ngram3.tsv
df_fasttext_predict_sub_epoch25_lr0.1_ngram4.tsv
df_fasttext_predict_sub_epoch50_lr0.05_ngram3.tsv
df_fasttext_predict_sub_epoch50_lr0.05_ngram4.tsv
df_fasttext_predict_sub_epoch50_lr0.1_ngram3_dim100_losshs.tsv


In [0]:
df_best=pd.read_csv('df_fasttext_predict_sub_epoch50_lr0.1_ngram3_dim100_lossns.tsv',sep='\t')
df_best.drop(labels=['Unnamed: 0'],axis=1,inplace=True)
df_best['MAIN']=df_best['CATEGORY_SUB'].map(lambda x: str(x).split('_')[0])
df_best['fasttext_MAIN']=df_best['fasttext_predict'].map(lambda x: str(x).split('_')[0])

In [42]:
df_best.head()

,CATEGORY_SUB,complaint,fasttext_predict,fasttext_complaint,MAIN,fasttext_MAIN
0,environment_garbage_collection,missed white goods pick mon two weeks refriger...,environment_garbage_collection,missed white goods pick mon two weeks refriger...,environment,environment
1,street_repair,pothole growing years many cars use driveway t...,street_repair,pothole growing years many cars use driveway t...,street,street
2,planning_general,neglected property eyesore overgrown vegetatio...,environment_overgrowth,neglected property eyesore overgrown vegetatio...,planning,environment
3,governance_general,stone school road north eisenhower recently pa...,street_general,stone school road north eisenhower recently pa...,governance,street
4,street_repair,potholes,street_repair,potholes,street,street


In [43]:
print('accuracy score:',accuracy_score(df_best.MAIN, df_best.fasttext_MAIN, normalize=True, sample_weight=None))
print(classification_report(df_best.MAIN,df_best.fasttext_MAIN))

accuracy score: 0.8842632615012863
                precision    recall  f1-score   support

   environemnt       0.44      0.35      0.39        54
   environment       0.93      0.95      0.94     99658
          fire       0.68      0.76      0.72      1080
    governance       0.55      0.49      0.52     10278
       housing       0.81      0.79      0.80     10367
infrastructure       0.78      0.76      0.77      7108
      planning       0.85      0.74      0.79      7515
  publichealth       0.91      0.91      0.91      5215
   publicorder       0.70      0.73      0.72      3441
        street       0.89      0.89      0.89     39904
     vandalism       0.90      0.92      0.91      7402

   avg / total       0.88      0.88      0.88    192022

